In [103]:
# import python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import csv
import math
import pickle
import random
import pprint

In [104]:
# Read user data
words = []
letters = []
first_index = 0
with open('data.txt', 'r') as input:
  """
  TODO: Parse user input data to a list of touch point collections/lists 
  """
  file = input.readlines()
  for line in file:
    if first_index == 0:
      first_index += 1
      continue
    elif line[0] == "=":
      words.append(letters)
      letters = []
    else:
      line_arr = line.split()
      line_arr_info = [line_arr[0], float(line_arr[1]), float(line_arr[2])]
      letters.append(line_arr_info)
  words.append(letters)
  pprint.pprint(words)


[[['i', 25.258852, 7.752883], ['f', 13.50101, 11.854456]],
 [['a', 4.9218874, 13.221646], ['t', 16.132853, 7.0692873]],
 [['f', 13.671909, 12.948208],
  ['i', 25.942448, 7.70731],
  ['r', 12.03128, 7.160434],
  ['s', 7.48537, 13.540659],
  ['t', 15.346718, 7.3427258]],
 [['y', 20.91802, 6.65913],
  ['o', 29.633863, 8.071894],
  ['u', 23.754942, 8.61877]],
 [['f', 15.244179, 12.67477],
  ['a', 4.648449, 13.084928],
  ['i', 25.703188, 7.4794445],
  ['l', 31.103592, 13.221646]],
 [['w', 5.6396623, 8.482051], ['e', 8.784202, 7.6617365]],
 [['r', 12.03128, 7.6161637],
  ['u', 24.711975, 7.7984557],
  ['n', 24.575256, 18.189108]],
 [['t', 15.551796, 8.390905],
  ['h', 21.054739, 13.768523],
  ['e', 9.262718, 8.254187]],
 [['r', 13.159212, 8.61877],
  ['i', 26.831121, 7.3427258],
  ['s', 7.3144712, 12.993782],
  ['k', 27.719795, 12.629196]],
 [['o', 29.97566, 6.385692], ['f', 14.458044, 13.039353]],
 [['f', 14.458044, 11.5810175],
  ['a', 5.4345837, 12.857063],
  ['i', 25.49811, 7.70731],
  [

In [105]:
# Read dictionary
with open('unigram.dict', 'rb') as unigramModelFile:
    unigramModel = pickle.load(unigramModelFile)
unigramModelFile.close()

# Read keyboard data
keyboard_raw = pd.read_csv("keyboard.csv")
keyboard = keyboard_raw[['key', 'x_mm', 'y_mm']]
keyboard
unigramModel

{'the': 0.03933837507090547,
 'of': 0.022362525338300483,
 'and': 0.022100157619537028,
 'to': 0.020636764209678228,
 'a': 0.015440912627459126,
 'in': 0.014400707674149294,
 'for': 0.010088551882990708,
 'is': 0.008001275333472512,
 'on': 0.006376923565241907,
 'that': 0.005781144503654221,
 'by': 0.005696158661744654,
 'this': 0.005489435157037871,
 'with': 0.0054123101306521575,
 'i': 0.0052475738476111455,
 'you': 0.005094469709217781,
 'it': 0.004783281792247097,
 'not': 0.00447777365836533,
 'or': 0.004405089635955221,
 'be': 0.004078601220057391,
 'are': 0.00406991378324621,
 'from': 0.0038692427175953605,
 'at': 0.003863593258032305,
 'as': 0.0038213555280641335,
 'your': 0.0035061751858210246,
 'all': 0.003438831081224539,
 'have': 0.002659646884637137,
 'new': 0.002637637746846681,
 'more': 0.002626607815112298,
 'an': 0.0025815408240069623,
 'was': 0.0025223050285674833,
 'we': 0.002364571742272109,
 'will': 0.002306134647147762,
 'home': 0.002171058640628696,
 'can': 0.0021

In [106]:
# Keyboard size and dual Gaussian model parameters
key_width = 3
key_height = 4
a = 2.403
b = 0.017
c = 2.295
d = 0.016

def get_likelihood(p, mu, sigma):
  """
  TODO: Calculate the likelihood that a touch point p is from the 2D Gaussian distribution N(mu, sigma)
  """
  lik = stats.multivariate_normal(mu, sigma).pdf(p)
  return lik

def is_letter(p, letter):
  """
  TODO: Determine if touch point p is located inside the boundary of the key: letter
  """
  letter_cord = []
  letter_cord.append(keyboard.loc[keyboard['key'] == letter, 'x_mm'].iloc[0])
  letter_cord.append(keyboard.loc[keyboard['key'] == letter, 'y_mm'].iloc[0])
  if p[0] >= letter_cord[0] and p[0] <= letter_cord[0] + key_width and p[1] >= letter_cord[1] and p[1] <= letter_cord[1] + key_height:
      return True
  return False  

def get_literal_string(touchpoints):
  """ 
  TODO: Compute the literal string using is_letter(p, letter) method for a collection of touch points that represents a word. 
        If a touch point does not fall inside any key boundary, use '?' to represent the corresponding character.
  """
  alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

  literal_string = ""
  for points in touchpoints:
    point_coords = [points[1], points[2]]
    q_mark = True
    for ch in alphabet:
      if is_letter(point_coords, ch):
        literal_string += ch
        q_mark = False
    if q_mark:
      literal_string += '?'
  return literal_string

# print(get_literal_string([[20.91802, 6.65913], [29.633863, 8.071894], [23.754942, 8.61877]]))

In [107]:
def unigram_lm_decoder(touchpoints):
  """
  A language decoder that uses the dual Gaussian touch point spatial disrtibution model and a unigram language model.
  Input: a list/collection of touch points that represents a certain word
  Output: the decoded word for the input

  TODO: Step a --- Get all possible words and their corresponding probabilities from the dictionary. 
        Use the length of the correct word to filter possible words
        You may also use the first and/or the last touchpoint to further narrow down possible words
  """
  alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
  possible_words = []
  if len(touchpoints) > 1:
    possible_first_keys = []
    possible_last_keys = []

    for letter in alphabet:
        rand = [keyboard.loc[keyboard['key']==letter, ['x_mm']]['x_mm'].values.tolist()[0] + key_width / 2, keyboard.loc[keyboard['key']==letter, ['y_mm']]['y_mm'].values.tolist()[0] + key_height / 2]
        if np.sqrt((touchpoints[0][1] - rand[0])**2 + (touchpoints[0][2] - rand[1])**2) < 1.5 * key_height:
            possible_first_keys.append(letter)

        if np.sqrt((touchpoints[-1][1] - rand[0])**2 + (touchpoints[-1][2] - rand[1])**2) < 1.5 * key_height:
            possible_last_keys.append(letter)
    
  for key, value in list(unigramModel.items()):
    if len(key) == len(touchpoints):
      if len(touchpoints) > 1:
        if key[0] in possible_first_keys and key[-1] in possible_last_keys:
          possible_words.append([key, value])
      else:
        possible_words.append([key, value])
  
  # Calculate p(w|s_1, s_2, ... s_n) ~ p(s_1, s_2, ..., s_n|w)*p(w) = \Pi(p(s_i|c_i))p(w) for each possible word
  p_w_s = []                  # Holds p(w|s_1, s_2, ..., s_n) for all possible words
  for item in possible_words:
    word = item[0]                   # TODO: The current possible word
    p_w = item[1]                    # TODO: Probability of the current possible word in the unigram language model
    p_s_w = 1                 # Holds p(s_1, s_2, ..., s_n|w) for the current possible word

    for j, letter in enumerate(list(word)):
      point = [touchpoints[j][1], touchpoints[j][2]]
      # TODO: Step b --- Apply the spatial model to get p(s_i|c_i)
      mu = [keyboard.loc[keyboard['key']==letter, ['x_mm']]['x_mm'].values.tolist()[0] + key_width/2, keyboard.loc[keyboard['key']==letter, ['y_mm']]['y_mm'].values.tolist()[0] + key_height/2] 
      sigma = [[a + b * key_width * key_width, 0], [0, c + d * key_height * key_height]] 
      p_s_c = get_likelihood(point, mu, sigma)

      # TODO: Step b --- Multiply the current p(s_i|c_i) to p(s_1, s_2, ..., s_n|W)
      p_s_w = p_s_w * p_s_c
    p_w_s.append((p_w)*(p_s_w))

    # TODO: Step c --- Calculate p(w|s_1, s_2, ... s_n) from p(s_1, s_2, ..., s_n|w) and p(w). Append the result to list
  max_probability = max(p_w_s) 
  max_index = p_w_s.index(max_probability)
  decoded_word = possible_words[max_index]

  # TODO: Step d ---- Choose word by the maximum of p(w|s_1, s_2, ..., s_n)
  return decoded_word

In [108]:
decoded_success_count = 0
literal_success_count = 0
decoded_words = []
literal_strings = []
correct_words = []

for touchpoints in words:
  """
  TODO: Use above methods to compute the correct word, decoded word, and the literal string for each touch point collection
        Append results to the corresponding list
        Update the decoded words/literal strings success count. 
          --- If the decoded word/literal string is the same as correct word, increase 1 to decoded words/literal strings success count
  """
  actual_str = ""
  for i in range(len(touchpoints)):
    actual_str += (touchpoints[i][0])
  correct_words.append(actual_str)
  literal_strings.append(get_literal_string(touchpoints))
  decoded_words.append(unigram_lm_decoder(touchpoints))
  print(decoded_words[-1][0], literal_strings[-1], len(decoded_words))
  if actual_str == literal_strings[-1]:
    literal_success_count += 1
  if actual_str == decoded_words[-1][0]:
    decoded_success_count += 1
  counter += 1
# TODO: calculate the success rate for both the decoded words and the literal strings using the docoded word/literal string success count
decoded_succ_rate = decoded_success_count / len(correct_words)
literal_succ_rate = literal_success_count / len(correct_words)

with open("results.txt", 'w') as output:
    output.write(str(decoded_succ_rate) + ", " + str(literal_succ_rate) + '\n')
    for i in range(len(correct_words)):
        output.write(str(correct_words[i]) + ", " + str(decoded_words[i][0]) + ", " + str(literal_strings[i]) + "\n")
        # Each line after: correct_word, decoded_word, literal_string
output.close()

of if 1
at at 2
first first 3
you upi 4
fail fai? 5
we we 6
run ri? 7
the tbe 8
risk rosk 9
of pf 10
failure fai?ire 11
watch qatchj 12
out put 13
for for 14
low ?ow 15
objects o?jects 16
please p?ease 17
provide pro?ode 18
your ypyr 19
date datw 20
circumference ?irci?asyancw 21
are ate 22
poor ?por 23
a a 24
problem ?rp?lem 25
with woth 26
the tbe 27
engine e?v?ne 28
my mt 29
favorite favoroye 30
subject sibject 31
elephants e?e?hants 32
are ate 33
large ??ehe 34
my my 35
favorite fabprite 36
place ?lacw 37
to to 38
visit ?ozot 39
can ??n 40
i i 41
skate skate 42
with wotb 43
sister sozyet 44
neither ?eotbwr 45
a a 46
borrower bprropwer 47
a a 48
question questop? 49
to tp 50
answer amzwet 51
three tbree 52
two tqo 53
one p?e 54
zero zerp 55
one p?e 56
heck hecl 57
of oc 58
a a 59
question qieztipm 60
the the 61
world wptld 62
is os 63
a a 64
stage syahe 65
prayer prayer 66
on om 67
schools scjppls 68
offends pccemd? 69
you ypu 70
must ?ust 71
be be 72
getting hetto?g 73
old p?d 74
h